# Pharaglow Analysis Notebook

In [1]:
# %matplotlib inline
%matplotlib qt

import logging
import numpy as np
import os
import pandas as pd
import shutil
import timeit
from datetime import date
# image io and analysis
import json
import pims
import trackpy as tp

# plotting
import matplotlib  as mpl 
import matplotlib.pyplot as plt 

#our packages
import pharaglow
from pharaglow import tracking, run, features, util, io



This notebook runs the pharaglow image analysis pipeline. It comprises three stages on analysis which can be done sequentially and are independent. Analyses can be interrupted at the end of each stage after saving the output dataframe. 

**1. Step -  Basic object detection**
    This step creates a "_features.json" file which contains a table of objects detected in each frame.
    Beyond finding the center of mass of an object, no further image analysis is done here.
    
**2. Step - Linking objects into trajectories**
    This results in individual files "_trajectory.json" for each tracked animal.
    
**3. Step - Analysing the details of object shapes**
    This step is doing the heavy lifting: It extracts centerlines, widths, contours and other object descriptors from the objects

All subsequent analyses steps add 'columns' to the data, and thus features is a subset of trajectories is a subset of results.

## 1. Setup

### Input parameters

In [2]:
## CHANGE FILE/DIRECTORY NAMES
# parameterfile = "../AnalysisParameters_1x"
# inPath = "/home/mscholz/Dropbox (Scholz Lab)/Shared/Data/MS0006_0_1000frames"
# outPath = "/home/mscholz/Desktop/TestOutput_MS0006/"
# lawnPath = None #"/opt/data/Lawns/"
# movie = "MS0006_0_1000frames"
# movieID = movie[-6:]
# nWorkers = 5
# chunksize = 100
# debug=False

parameterfile = "/home/ebonnard/Desktop/Elsa/1_Rawdata/20210301_macroscope/GGE_parameters.txt"
inPath = "/home/ebonnard/Desktop/Elsa/1_Rawdata/20210301_macroscope/MA0001c/"
outPath = "/home/ebonnard/Desktop/Elsa/2_PharaGlow/20210301_macroscope/"
lawnPath = None #"/opt/data/Lawns/"
movie = "MA0001"
movieID = "MA0001"
nWorkers = 10
chunksize = 100
debug=True

# # Write the path of the directories where
# # > is the parameters file
# parameterfile = '/home/ebonnard/Desktop/Elsa/2_PharaGlow/20201217_solenoid_mutants/EB0068/Linking/EB0068_parameters.txt'
# # > are the tiff files
# inPath = '/home/nif/Desktop/data/Elsa/1_Rawdata/20201217_solenoid_mutants/recording/EB0068c/'
# # > will be saved the pharaglow output files
# outPath = f'/home/ebonnard/Desktop/Elsa/2_PharaGlow/20201217_solenoid_mutants/EB0068/broken_pharynx/'
# # > is the tiff file with the bacterial lawn (if no lawn: None)
# lawnPath = None #"/opt/data/Lawns/"

# # Set the names of the tiff files folder (movie) and the identification number (movieID) of the recording
# movie = "EB0068"
# movieID = movie # the ID should be AA1000
# # movieID = movie[-6:]

# # Set the number of processing cores used for the analysis
# nWorkers = 10
# chunksize = 100

# # Inactivate (False) or activate (True) the debug mode
# debug=False

#### Add a logger

In [3]:
today = date.today().strftime("%Y%m%d")
logger = io.log_setup('PharaGlow', 10, fname = os.path.join(outPath, f'{today}_{movieID}_pharaglow_log.txt')   )
logging.debug('pharaglow')

#### Check paths

In [4]:
npaths = {'parameter file': parameterfile,
          'inPath':inPath,
          'outPath': outPath}

for key, value in npaths.items():    
    if os.path.exists(value):
        print(f'{key}: {value}')
    else:
        print(f"Warning! The path for {key} doesnt exist: {value}")

parameter file: /home/ebonnard/Desktop/Elsa/1_Rawdata/20210301_macroscope/GGE_parameters.txt
inPath: /home/ebonnard/Desktop/Elsa/1_Rawdata/20210301_macroscope/MA0001c/
outPath: /home/ebonnard/Desktop/Elsa/2_PharaGlow/20210301_macroscope/


#### Creating new file names

In [5]:
fname = os.path.join(inPath,"*.tif*")
outfile = os.path.join(outPath, movieID+"_{}_{}.json")
logger.info(f"output file will be saved as {outfile}")
saveparam = os.path.join(outPath, movieID+"_parameters")
logger.info(f"paramaters file will be saved as {saveparam}")

INFO:PharaGlow:output file will be saved as /home/ebonnard/Desktop/Elsa/2_PharaGlow/20210301_macroscope/MA0001_{}_{}.json
INFO:PharaGlow:paramaters file will be saved as /home/ebonnard/Desktop/Elsa/2_PharaGlow/20210301_macroscope/MA0001_parameters


### Load data

In [6]:
if debug:
    
    logger.debug('Select tiff files to analyze:')
    first_tiff = 1 # the tiff file name/number must be >= 1
    last_tiff = 800

    n = np.arange(first_tiff-1, last_tiff)  
    logger.debug(f'first tiff:{first_tiff}, n.min:{n.min()}')
    logger.debug(f'last tiff:{last_tiff}, n.max:{n.max()}')

DEBUG:PharaGlow:Select tiff files to analyze:
DEBUG:PharaGlow:first tiff:1, n.min:0
DEBUG:PharaGlow:last tiff:800, n.max:799


##### Load lawns if available

In [7]:
if lawnPath is not None and lawnPath != 'None':
    logger.info('Loading lawn file...')
    try:
        lawnfile = os.path.join(lawnPath,movieID+'_lawn.tiff')
        lawn = pims.open(lawnfile)[0]
        binLawn = features.findLawn(lawn)
    except Exception:
        lawnfile = os.path.join(lawnPath,movieID+'_lawn.bmp')
        lawn = pims.open(lawnfile)[0]
        binLawn = features.findLawn(lawn)
    logger.info("Lawnfile opened as 'lawn'")
else:
    lawnfile = None

##### Load images and analysis parameters

In [8]:
start = timeit.default_timer()
logger.info("Loading tiff files...")
rawframes = pims.open(fname)

if not debug:
    rawframes = rawframes
if debug:
    logger.debug(f"A subset of {len(n)} files will be analyzed")
    rawframes = rawframes[n]
logger.info("tiff files loaded as 'rawframes'")


logger.info(f"Loading parameters from {parameterfile}...")
with open(parameterfile) as f:
    param = json.load(f)
    f.close()
logger.info(f"parameters file loaded as 'param':{param}")

# Measure the wall time for running the current cell [s]
stop = timeit.default_timer()
logger.info(f"Loading time: {stop - start}s")  

INFO:PharaGlow:Loading tiff files...
DEBUG:PharaGlow:A subset of 800 files will be analyzed
INFO:PharaGlow:tiff files loaded as 'rawframes'
INFO:PharaGlow:Loading parameters from /home/ebonnard/Desktop/Elsa/1_Rawdata/20210301_macroscope/GGE_parameters.txt...
INFO:PharaGlow:parameters file loaded as 'param':{'subtract': 1, 'smooth': 1, 'dilate': 2, 'tfactor': 1, 'thresholdWindow': 120, 'bgWindow': 100, 'length': 2, 'searchRange': 10, 'minimalDuration': 600, 'memory': 30, 'minSize': 600, 'maxSize': 1500, 'watershed': 100, 'widthStraight': 10, 'pad': 5, 'nPts': 200, 'linewidth': 2}
INFO:PharaGlow:Loading time: 0.20721589800086804s


#### Check if all the tiff files have been loaded as rawframes

In [9]:
nfiles = len([f for f in os.listdir(inPath) if f.endswith('tif')])
# tiff files
logger.info(f"Number of tiff files: {nfiles}")
# rawframes 
logger.info(f"Number of rawframes: {len(rawframes)}")

if nfiles != len(rawframes):
    if not debug:
        logger.warning("the number of tiff files doesn't match with the number of rawframes !")


INFO:PharaGlow:Number of tiff files: 0
INFO:PharaGlow:Number of rawframes: 800


### Improve lawn detection if neccessary

In [10]:
if lawnfile is not None:
    from skimage.filters import threshold_li, gaussian, threshold_yen, threshold_otsu
    from skimage.morphology import skeletonize, watershed, disk, remove_small_holes, remove_small_objects
    image = gaussian(lawn, 1, preserve_range = True)
    thresh = threshold_li(image, initial_guess = np.median)
    binary = image > thresh*0.5
    binary = remove_small_holes(binary, area_threshold=1500, connectivity=1, in_place=False)
    binary = remove_small_objects(binary, min_size=5000, connectivity=8, in_place=False)
    binLawn = binary
    plt.figure(figsize=(12,6))
    plt.subplot(121)
    plt.imshow(lawn)
    plt.contour(binLawn, alpha=0.5, cmap='pink')
    plt.subplot(122)
    plt.imshow(binLawn)

## 2. Object detection

### Create binary masks

In [25]:
## To quickly adjust the parameters
# param['bgWindow']=71
# logger.debug(f"parameter bgWindow changed to {param['bgWindow']}")

# param['thresholdWindow']=111
# logger.debug(f"parameter thresholdWindow changed to {param['thresholdWindow']}")

# param['smooth']=1
# logger.debug(f"parameter smooth changed to {param['smooth']}")

# param['dilate']=2
# logger.debug(f"parameter dilate changed to {param['dilate']}")

# param['tfactor'] = 1
# logger.debug(f"parameter tfactor changed to {param['tfactor']}")


DEBUG:PharaGlow:parameter tfactor changed to 1


In [26]:
start = timeit.default_timer()

# detecting objects
logger.info('Binarizing images...')

masks = tracking.calculateMask(rawframes,
                               minSize = param['minSize'],
                               bgWindow = param['bgWindow'],
                               thresholdWindow = param['thresholdWindow'],
                               smooth =  param['smooth'],
                               subtract =  param['subtract'],
                               dilate = param['dilate'],
                               tfactor=param['tfactor'])


stop = timeit.default_timer()
logger.info(f"binary masks created ({stop - start}s)")  

INFO:PharaGlow:Binarizing images...
INFO:PharaGlow:binary masks created (3.3504359490034403s)


### Make sure the thresholding worked otherwise change parameters

In [27]:
# Select a rawframe to visualize
t = 539

if t> (len(rawframes)-1):
    # Check if the selected rawframe is present otherwise select the first frame
    print(f"Warning ! Max {len(rawframes)-1} rawframes. {t} changed to 0")
    t=0

if debug:
    if first_tiff:
        print(f"rawframe {t} (or tiff {first_tiff+t}) to visualize")      
else:
    print(f"rawframe {t} to visualize ")

rawframe 539 (or tiff 540) to visualize


##### Visualize the raw data

In [28]:
plt.figure(figsize=(16,8))

plt.subplot(121)
# Plot the histogram of the pixel intensity values of the rawframe
plt.hist(rawframes[t].ravel(), bins=256, log=True)
plt.xlim(0, 260) # xlim for 8 bits image

plt.subplot(122)
# Adjust the color limit for a better vizualisation of the rawframe
# color = (0,150) # 0<=color<=255 for 8 bits image
color = None 
plt.imshow(rawframes[t],clim = color)
plt.colorbar(orientation='horizontal');

# plt.savefig(os.path.join(outPath,f'{today}_{movieID}_px_hist_{t}.pdf'))

##### Show the mask and detected objects

In [29]:
# %matplotlib qt

from skimage.measure import label, regionprops

plt.figure(figsize=(18,14))
plt.subplot(121)
# Show the rawframe
plt.imshow(rawframes[t],clim= color)#+lawn)
if lawnfile is not None:
    # Show the lawn
    plt.contour(binLawn, alpha=0.5, cmap='pink')
    
plt.subplot(122)
# Show the masks and their size [px]
plt.imshow(masks[t])#[:600,1000:])#[500:1500,2000:3500])#[:,2500:])
# print(np.min(masks[t]))
label_image, num = label(masks[t], background=0, connectivity = 1,return_num=True)
print(f"{num} detected objects")
for region in regionprops(label_image):
    plt.text(region.centroid[1]+50, region.centroid[0], region.area, color ='w')

    
plt.tight_layout()

# if not debug:
#     plt.title(f"Rawframe and masks (#{num}) at t={t} ({movieID})",fontsize=24)
#     plt.savefig(os.path.join(outPath,f'{today}_{movieID}_masks_rawframe{t}.pdf'))
if debug:
    plt.title(f"Rawframe and masks (#{num}) at {t} (tiff {first_tiff+t}) ({movieID})", fontsize=24)
    plt.savefig(os.path.join(outPath,f'{today}_{movieID}_masks_tiff{first_tiff+t}.pdf'))

2 detected objects


### Detecting individual objects and tracking or use multiprocessing to speed up feature detection

In [30]:
logger.info(f'chunksize={chunksize}')

INFO:PharaGlow:chunksize=100


In [40]:
# param['maxSize']=2000 #1500  2000  1800  1900
# logger.debug(f"parameter maxSize changed to {param['maxSize']}")

# param['watershed'] = 20 # 100  70  20
# logger.debug(f"parameter watershed changed to {param['watershed']}")

DEBUG:PharaGlow:parameter maxSize changed to 2000
DEBUG:PharaGlow:parameter watershed changed to 20


In [41]:
start = timeit.default_timer()

logger.info('Detecting features...')

if nWorkers ==1 or len(rawframes) < chunksize:
    logger.info('...without parallel detection...')
    features, images = tracking.runfeatureDetection(rawframes, masks, param, frameOffset = 0)
    images.columns = [f"im{s}" for s in images.columns]
    features = pd.concat([features, images], axis = 1)
else:
    logger.info('...with parallel detection...')
    features = tracking.parallelDetection(rawframes, masks, param, nWorkers, chunksize)
    

stop = timeit.default_timer()
logger.info(f"features detected ({stop - start}s)")  

INFO:PharaGlow:Detecting features...
INFO:PharaGlow:...with parallel detection...


Analyzing frames 0 to 100
Analyzing frames 100 to 200
Analyzing frames 200 to 300
Analyzing frames 300 to 400
Analyzing frames 400 to 500
Analyzing frames 500 to 600


/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/thresholding.py:638: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))


Analyzing frames 600 to 700


/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/thresholding.py:638: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))


Analyzing frames 700 to 800


INFO:PharaGlow:features detected (306.55987730300694s)


In [42]:
# Files monitoring
logger.info(f" Number of frames in features:{features['frame'].nunique()}")
                                                       
if len(rawframes) != len(features['frame'].unique()):
    logger.warning(f" Number of frames in features ({features['frame'].nunique()}) and the number of rawframes ({len(rawframes)}) don't match !")

INFO:PharaGlow: Number of frames in features:800


#### Visualize results of object detection

In [43]:
### Show the area of all objects
plt.figure(figsize=(12,6))
plt.subplot(121)
features['area'].hist(bins = 30)
plt.xlabel('Area (px)')
plt.subplot(122)
features['frame'].value_counts().sort_index().plot()
plt.ylabel('Number of objects')
plt.xlabel('Frame')
#features['frame'].hist(bins = len(rawframes))

logger.info(f"features.area.min():{features.area.min()}") # region.area > params['minSize']
logger.info(f"features.area.max():{features.area.max()}") # region.a#     t=30rea < params['maxSize'] 

plt.savefig(os.path.join(outPath,f'{today}_{movieID}_features_objects.pdf'))

INFO:PharaGlow:features.area.min():451
INFO:PharaGlow:features.area.max():1936


### Save features

In [45]:
start = timeit.default_timer()

# saving features
logger.info("Saving features...")
features.info(memory_usage='deep')
features.to_json(outfile.format('features', 'all'), orient='split')
stop = timeit.default_timer()
logger.info(f"features saved as {outfile.format('features', 'all')} ({stop - start}s)")

# saving parameter file
logger.info("Saving parameters...")
shutil.copyfile(parameterfile, saveparam, follow_symlinks=True)
logger.info(f"parameters saved as {parameterfile}")


INFO:PharaGlow:Saving features...
INFO:PharaGlow:features saved as /home/ebonnard/Desktop/Elsa/2_PharaGlow/20210301_macroscope/MA0001_features_all.json (1.249721532993135s)
INFO:PharaGlow:Saving parameters...
INFO:PharaGlow:parameters saved as /home/ebonnard/Desktop/Elsa/1_Rawdata/20210301_macroscope/GGE_parameters.txt


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2316 entries, 0 to 2315
Columns: 5487 entries, y to im5474
dtypes: float64(579), int64(8), uint8(4900)
memory usage: 21.2 MB


#### (Optional) Load features if continuing prior analysis

In [46]:
# %%time  
# # Load feature
# start = timeit.default_timer()
# logger.info("Loading features...")
# features = io.load(outfile.format('features', 'all'), orient='split')
# features.info()
# stop = timeit.default_timer()
# logger.info(f"features loaded ({stop - start}s)")

## 3. Creating trajectories

### Link objects to trajectories using trackpy and interpolate short misses

In [47]:
logger.info('Linking trajectories...')
logger.info(f"Parameter searchRange: {param['searchRange']} px")
logger.info(f"Parameter memory: {param['memory']} frames")

INFO:PharaGlow:Linking trajectories...
INFO:PharaGlow:Parameter searchRange: 10 px
INFO:PharaGlow:Parameter memory: 30 frames


In [48]:
#pred = tp.predict.NearestVelocityPredict()
#trajectories = pred.link_df(features,param['searchRange'], memory = param['memory'])
trajectories = tp.link_df(features,param['searchRange'], memory = param['memory'])
logger.info(f"Number of trajectories after linking: {len(trajectories['particle'].unique())}")

INFO:PharaGlow:Number of trajectories after linking: 10


Frame 799: 4 trajectories present.


In [ ]:
# Delete the variable features to save memory
del features

In [ ]:
if debug:
    plt.figure(figsize=(8,8))
    tp.plot_traj(trajectories, colorby = 'particle', superimpose=1-masks[t],label=False)
    plt.savefig(os.path.join(outPath,f'{today}_{movieID}_trajectories_after_linking.pdf'))

In [49]:
logger.info('Interpolating trajectories...')

traj = []

for particle_index in trajectories['particle'].unique():
    tmp = trajectories[trajectories.loc[:,'particle'] == particle_index]
    # interpolate data but do not interpolate the images!
    tmp = tracking.interpolateTrajectories(tmp, columns = ['x', 'y', 'shapeX', 'shapeY', 'particle'])
    # replace nans again and convert images to uints8
    imcols = [col for col in tmp.columns if "im" in str(col)]
    tmp[imcols] = tmp[imcols].fillna(0).astype('uint8')
    traj.append(tmp)
trajectories = pd.concat(traj, ignore_index = True)
trajectories['shapeX'] = trajectories['shapeX'].astype(int)
trajectories['shapeY'] = trajectories['shapeY'].astype(int)

logger.info(f"Interpolation done.")

INFO:PharaGlow:Interpolating trajectories...
INFO:PharaGlow:Interpolation done.


In [51]:
if debug:
    plt.figure(figsize=(8,8))
    tp.plot_traj(trajectories, colorby = 'particle', superimpose=1-masks[t],label=False)
    plt.savefig(os.path.join(outPath,f'{today}_{movieID}_trajectories_after_interpolation.pdf'))

In [ ]:
# param['minimalDuration'] = 60 # 900
# logger.debug(f"parameter minimalDuration changed to {param['minimalDuration']}")

In [52]:
logger.info(f"Filtering out trajectories which last less than the minimal duration ({param['minimalDuration']} frames)...")
logger.info(f"Nb of trajectories before filtering: {trajectories['particle'].nunique()}")

trajectories = tp.filter_stubs(trajectories,param['minimalDuration'])
logger.info(f"Nb of trajectories after filtering: {trajectories['particle'].nunique()}")

INFO:PharaGlow:Filtering out trajectories which last less than the minimal duration (600 frames)...
INFO:PharaGlow:Nb of trajectories before filtering: 10
INFO:PharaGlow:Nb of trajectories after filtering: 2


#### Extract lawn info

In [54]:
def inside(x,y,binLawn):
    return binLawn[int(y), int(x)]

if lawnfile is not None:
    trajectories['inside'] = trajectories.apply(\
        lambda row: pd.Series(inside(row['x'], row['y'], binLawn)), axis=1)

#### Show the trajectories

In [55]:
if debug:
    print("Show resulting trajectories")
    plt.figure(figsize=(11,11))
    plt.title(f"{trajectories['particle'].nunique()} trajectories detected ({movieID})")
    tp.plot_traj(trajectories)
    # tp.plot_traj(trajectories, superimpose=1-masks[t], label=False)

    plt.savefig(os.path.join(outPath,f'{today}_{movieID}_resulting_trajectories.pdf'))  

Show resulting trajectories


#### Save individual trajectories & add the missing images to interpolated trajectories

In [56]:
start = timeit.default_timer()

# # write trajectories to separate files.

logger.info(f"Saving {trajectories['particle'].nunique()} trajectories to separate files...")

for particle_index in trajectories['particle'].unique():
    tmp = trajectories[trajectories.loc[:,'particle'] == particle_index]
    # fill missing images now
    tmp = tmp.apply(\
    lambda row: tracking.interpolate_helper(rawframes, row, param), axis=1)
    # replace nans again and convert images to uints8
    imcols = [col for col in tmp.columns if "im" in str(col)]
    tmp[imcols] = tmp[imcols].fillna(0).astype('uint8')
    # write trajectories to file
    tmp.to_json(outfile.format('trajectories', int(particle_index)), orient='split')
    
stop = timeit.default_timer()
logger.info(f"trajectories saved as json files ({stop - start}s)") 


INFO:PharaGlow:Saving 2 trajectories to separate files...


interpolating 493.0
interpolating 496.0
interpolating 500.0
interpolating 501.0
interpolating 502.0
interpolating 503.0
interpolating 505.0
interpolating 507.0
interpolating 508.0
interpolating 509.0
interpolating 510.0
interpolating 512.0
interpolating 513.0
interpolating 514.0
interpolating 515.0
interpolating 516.0
interpolating 520.0
interpolating 521.0
interpolating 523.0
interpolating 526.0
interpolating 528.0
interpolating 530.0
interpolating 534.0
interpolating 535.0
interpolating 536.0
interpolating 537.0
interpolating 538.0
interpolating 539.0
interpolating 540.0
interpolating 541.0
interpolating 542.0
interpolating 544.0
interpolating 545.0
interpolating 549.0
interpolating 550.0
interpolating 552.0
interpolating 553.0
interpolating 557.0
interpolating 558.0
interpolating 561.0
interpolating 562.0
interpolating 564.0
interpolating 565.0
interpolating 566.0
interpolating 567.0
interpolating 568.0
interpolating 569.0


INFO:PharaGlow:trajectories saved as json files (32.888778225998976s)


### check slow-down before continuing analysis

In [57]:
if lawnfile is not None:
    plt.figure(figsize=(12,8))
    vcut = []
    dt = 1000
    for pid in trajectories['particle'].unique():
        tmp = trajectories[['frame', 'x', 'y']][trajectories.loc[:,'particle'] == pid].diff()
        f = (trajectories[['inside']][trajectories.loc[:,'particle'] == pid]).mean().values
        if f<0.9 and f>0.01:
            t0 = np.where((trajectories[['inside']][trajectories.loc[:,'particle'] == pid])==1)[0][0]
            print('t0:', t0)
            try:
                if t0>dt:
                    print('pid:', pid)
                    time = np.linspace(0,2*dt/30., 2*dt)
                    #print('time:', len(time))
                    v = np.sqrt((tmp['x']**2+tmp['y']**2))/tmp['frame']*30*2.4
                    #print('v:', v)
                    #print('v.iloc:', v.iloc[t0-dt:t0+dt])
                    plt.plot(time, v.iloc[t0-dt:t0+dt], 'navy', alpha=0.1)
                    vcut.append(v.iloc[t0-dt:t0+dt].values)
                else:
                    print('trajectory is too short')
            except ValueError:
                print('t0-dt or t0+dt exceeds number of frames')
                continue
                    
    if len(vcut) >0:  
        plt.plot(time, np.mean(np.array(vcut), axis=0), color='navy')
        plt.plot(time, util.smooth(np.mean(np.array(vcut), axis=0), 30), color='r')
        plt.axvline(dt/30, color='k', linestyle='--')
        plt.ylabel(r"velocity ($\mu$m/s)");
        plt.xlabel("time (s)");
        plt.ylim(0,150)

## 3. Run the whole pharaglow feature extraction

In [58]:
start = timeit.default_timer()

# save only minimal outputs - reduces save by approx factor 3
save_minimal = True
path = os.path.dirname(outfile)

for fn in os.listdir(path):
    file = os.path.join(path,fn)
    if os.path.isfile(file) and f'{movieID}_trajectories_' in fn and fn.endswith('.json'):
        particle_index = int(fn.split('.')[0].split('_')[-1])
        traj =  io.load(file, orient='split')
        # create a temporary column with an image array
        traj['image'] = traj.apply(lambda row: util.get_im(row, [f'im{i}' for i in range(int(row['shapeX'])*int(row['shapeY']))],
                                            int(row['shapeX'])), axis=1)
        
        # skip invalid frames
        invalid_images = traj[[np.sum(im)==0 for im in traj['image']]].index
        if len(invalid_images)>0:
            logger.info(f'invalid images in frames {invalid_images}')
        # ignore rows where images are empty - we later will interpolate
        traj = traj.drop(invalid_images)
        if len(traj.index)<1:
            print('Skipped', file)
            continue
        traj['shapeX'] = traj['shapeX'].astype(int)
        print('Analyzing trajectory:', fn)
        tmp = util.parallelize_dataframe(traj, run.runPharaglowOnStack, n_cores = nWorkers, params = param)
        
        # get more exact entry location
        if lawnfile is not None:
            tmp['insideHead'] = tmp.apply(\
                lambda row: pd.Series(features.headLocationLawn(row['Centerline'],(row['slice_y0'], row['slice_x0']), binLawn)), axis=1)
            tmp['insideHeadIntensity'] = tmp.apply(\
                lambda row: pd.Series(features.headLocationLawn(row['Centerline'],(row['slice_y0'], row['slice_x0']), lawn)), axis=1)
        
        # remove some columns to make the result smaller
        if save_minimal:
            tmp = tmp.drop(['Mask', 'SkeletonX', 'SkeletonY', 'ParX', 'ParY', 
                            'Xstart', 'Xend', 'Centerline', 'dCl', 'Widths', 'Contour', 'Gradient', 
                            'Kymo', 'KymoGrad', 'Similarity', 'Xtmp'], axis = 1)
        
        tmp = tmp.drop(['image'], axis = 1)
        tmp.to_json(outfile.format('results', particle_index), orient='split')

if save_minimal:
    logger.info('minimal information saved')
    
stop = timeit.default_timer()
logger.info(f"Whole pharaglow features extracted ({stop - start}s)")  

Analyzing trajectory: MA0001_trajectories_1.json
[62, 62, 62, 62, 62, 62, 62, 61, 61, 61]


/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/thresholding.py:638: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/thresholding.py:638: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/thresholding.py:638: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/threshol

Analyzing trajectory: MA0001_trajectories_0.json
[80, 80, 80, 80, 80, 80, 80, 80, 80, 80]


/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/thresholding.py:638: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/skimage/filters/thresholding.py:638: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/opt/anaconda/envs/pumping/lib/python3.7/site-packages/scipy/optimize/minpack.py:

#### Check if data has been analyzed

In [59]:
# Files monitoring

files_list = os.listdir(outPath)
f1 =[]
f2 =[]

for fn in files_list:
    file = os.path.join(outPath,fn)
    if os.path.isfile(file) and f'{movieID}_trajectories_' in fn  and fn.endswith('.json'):
        if not 'all' in fn: 
            particle_index = int(fn.split('.')[0].split('_')[-1])
            f1.append(particle_index)
    if os.path.isfile(file) and f'{movieID}_results_' in fn and fn.endswith('.json'): 
        particle_index = int(fn.split('.')[0].split('_')[-1])
        f2.append(particle_index)


logger.info('trajectories.json files: %s ', len(f1))
logger.info('results.json files: %s ', len(f2))
if len(f1) != len(f2):
    logger.warning('trajectories - results: %s', set(f1).symmetric_difference(set(f2)))


INFO:PharaGlow:trajectories.json files: 2 
INFO:PharaGlow:results.json files: 2 


In [60]:
# Saving parameters if they have been changed (debug mode activated)
if debug:
    logger.debug(f"New parameters:{param}")
    paramPath = os.path.join(outPath, movieID + '_parameters_new.txt')
    with open(paramPath,'w') as f:
         f.write(json.dumps(param)) # use `json.loads` to do the reverse
         
    logger.debug(f"New parameters saved as {paramPath}")

DEBUG:PharaGlow:New parameters:{'subtract': 1, 'smooth': 1, 'dilate': 2, 'tfactor': 1, 'thresholdWindow': 120, 'bgWindow': 100, 'length': 2, 'searchRange': 10, 'minimalDuration': 600, 'memory': 30, 'minSize': 600, 'maxSize': 2000, 'watershed': 20, 'widthStraight': 10, 'pad': 5, 'nPts': 200, 'linewidth': 2}
DEBUG:PharaGlow:New parameters saved as /home/ebonnard/Desktop/Elsa/2_PharaGlow/20210301_macroscope/MA0001_parameters_new.txt


In [61]:
logger.info("PharaGlow ends here")

INFO:PharaGlow:PharaGlow ends here
